# Ensemble methods. Exercises


In this section we have only two exercise:

1. Find the best three classifier in the stacking method using the classifiers from scikit-learn package.

2. Build arcing arc-x4 method. 

In [241]:
%store -r data_set
%store -r labels
%store -r test_data_set
%store -r test_labels
%store -r unique_labels

## Exercise 1: Find the best three classifier in the stacking method

Please use the following classifiers:

* Linear regression,
* Nearest Neighbors,
* Linear SVM,
* Decision Tree,
* Naive Bayes,
* QDA.

In [242]:
import numpy as np
from sklearn.metrics import accuracy_score
import itertools

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [243]:
def build_classifiers():
    return [
        ['linear regression', LinearRegression().fit(data_set, labels)],
        ['kneighbors', KNeighborsClassifier().fit(data_set, labels)],
        ['svc', SVC().fit(data_set, labels)],
        ['descision tree', DecisionTreeClassifier().fit(data_set, labels)],
        ['gaussian nb', GaussianNB().fit(data_set, labels)],
        ['quatratic discriminant', QuadraticDiscriminantAnalysis().fit(data_set, labels)]
    ]

In [244]:
def build_stacked_classifier(classifiers):
    output = []
    for classifier in classifiers:
        output.append(classifier.predict(data_set))
    output = np.array(output).reshape((130,3))
    
    # stacked classifier part:
    stacked_classifier = DecisionTreeClassifier()
    stacked_classifier.fit(output.reshape((130,3)), labels.reshape((130,)))
    test_set = []
    for classifier in classifiers:
        test_set.append(classifier.predict(test_data_set))
    test_set = np.array(test_set).reshape((len(test_set[0]),3))
    predicted = stacked_classifier.predict(test_set)
    return predicted

In [245]:
classifiers = build_classifiers()

best_acc = 0
best_comb = []

for combination in itertools.combinations(classifiers, 3):
    predicted = build_stacked_classifier([el[1] for el in combination])
    accuracy = accuracy_score(test_labels, predicted)
    if accuracy > best_acc:
        best_acc = accuracy
        best_comb = combination
        
print("Best classifiers:")
print(best_acc, [el[0] for el in best_comb])
        

Best classifiers:
0.95 ['svc', 'descision tree', 'quatratic discriminant']


C:\Users\konik\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


## Exercise 2: 

Use the boosting method and change the code to fullfilt the following requirements:

* the weights should be calculated as:
$w_{n}^{(t+1)}=\frac{1+ I(y_{n}\neq h_{t}(x_{n})}{\sum_{i=1}^{N}1+I(y_{n}\neq h_{t}(x_{n})}$,
* the prediction is done with a voting method.

In [348]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier

# prepare data set

def generate_data(sample_number, feature_number, label_number):
    data_set = np.random.random_sample((sample_number, feature_number))
    labels = np.random.choice(label_number, sample_number)
    return data_set, labels

labels = 2
dimension = 2
test_set_size = 1000
train_set_size = 5000
train_set, train_labels = generate_data(train_set_size, dimension, labels)
test_set, test_labels = generate_data(test_set_size, dimension, labels)

# init weights
number_of_iterations = 10
weights = np.ones((test_set_size,)) / test_set_size


def train_model(classifier, weights):
    return classifier.fit(X=test_set, y=test_labels, sample_weight=weights)

Fill the two functions below:

In [349]:
wrong_class = np.zeros(len(test_labels))

def set_new_weights(model):
    global wrong_class
    wrong_class += calculate_error(model)

    new_weights = (1 + wrong_class)
    new_weights /= sum(new_weights)
    return new_weights

def calculate_error(model):
    predicted = model.predict(test_set)
    return predicted != test_labels

Train the classifier with the code below:

In [350]:
classifier = DecisionTreeClassifier(max_depth=1, random_state=1)
classifier.fit(X=train_set, y=train_labels)
alphas = []
classifiers = []
for iteration in range(number_of_iterations):
    model = train_model(classifier, weights)
    weights = set_new_weights(model)
    classifiers.append(model)

print(weights)


validate_x, validate_label = generate_data(1, dimension, labels)

[0.00084746 0.00101695 0.00101695 0.00084746 0.00118644 0.00101695
 0.00084746 0.00118644 0.00101695 0.00101695 0.00118644 0.00101695
 0.00101695 0.00118644 0.00084746 0.00084746 0.00101695 0.00084746
 0.00084746 0.00101695 0.00084746 0.00101695 0.00118644 0.00084746
 0.00101695 0.00101695 0.00101695 0.00101695 0.00101695 0.00101695
 0.00084746 0.00084746 0.00101695 0.00101695 0.00118644 0.00101695
 0.00101695 0.00101695 0.00101695 0.00084746 0.00101695 0.00101695
 0.00084746 0.00118644 0.00118644 0.00101695 0.00101695 0.00101695
 0.00084746 0.00101695 0.00118644 0.00084746 0.00084746 0.00101695
 0.00084746 0.00118644 0.00084746 0.00118644 0.00084746 0.00101695
 0.00101695 0.00101695 0.00101695 0.00084746 0.00084746 0.00118644
 0.00101695 0.00084746 0.00101695 0.00118644 0.00101695 0.00084746
 0.00084746 0.00118644 0.00101695 0.00084746 0.00101695 0.00101695
 0.00084746 0.00118644 0.00101695 0.00084746 0.00084746 0.00118644
 0.00101695 0.00101695 0.00084746 0.00101695 0.00101695 0.0010

Set the validation data set:

In [351]:
validate_x, validate_label = generate_data(1, dimension, labels)

Fill the prediction code:

In [352]:
def get_prediction(x):
    output = []
    for classifier in classifiers:
        output.append(classifier.predict(validate_x))
    output = np.array(output)

    predicted = []
    for i in range(len(x)):
        classified = output[:, i]
        counts = np.bincount(classified)
        predicted.append(np.argmax(counts))
    return predicted


Test it:

In [353]:
prediction = get_prediction(validate_x)[0]
print(prediction, validate_label)

0 [0]
